<a href="https://colab.research.google.com/github/atulkakrana/Data-Analytics/blob/master/ML_Foundations_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from tensorflow.keras import layers, activations, models, metrics
from tensorflow.keras import losses, optimizers, callbacks, utils
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.3.0


In [6]:
import os, zipfile, random
from shutil import copyfile
import numpy as np

## Helper Functions

In [7]:
def plots(history):
  plt.xlabel('Epochs ->')
  plt.ylabel("Loss")
  plt.plot(history.history['loss'])

  return plt

def loss_acc_plots(history):
  '''
  Generate 
  '''
  acc      = history.history['accuracy']
  val_acc  = history.history['val_accuracy']
  loss     = history.history['loss']
  val_loss = history.history['val_loss']

  epochs   = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

  return None

In [8]:
## Get Data
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2020-09-16 03:14:39--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 92.123.124.137, 2a02:26f0:6b:5a5::e59, 2a02:26f0:6b:5b3::e59
Connecting to download.microsoft.com (download.microsoft.com)|92.123.124.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   128MB/s    in 7.6s    

2020-09-16 03:14:46 (104 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [9]:
print(len(os.listdir("/tmp/PetImages/Cat")))
print(len(os.listdir("/tmp/PetImages/Dog")))

12501
12501


In [10]:
try:
  os.mkdir('/tmp/cats-v-dogs')
  os.mkdir('/tmp/cats-v-dogs/train')
  os.mkdir('/tmp/cats-v-dogs/test')
  os.mkdir('/tmp/cats-v-dogs/train/cats')
  os.mkdir('/tmp/cats-v-dogs/train/dogs')
  os.mkdir('/tmp/cats-v-dogs/test/cats')
  os.mkdir('/tmp/cats-v-dogs/test/dogs')
except OSError:
  pass

In [14]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length  = int(len(files) - training_length)
    shuffled_set    = random.sample(files, len(files))
    training_set    = shuffled_set[0:training_length]
    testing_set     = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file   = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file   = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR    = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/train/cats/"
TESTING_CATS_DIR  = "/tmp/cats-v-dogs/test/cats/"
DOG_SOURCE_DIR    = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/train/dogs/"
TESTING_DOGS_DIR  = "/tmp/cats-v-dogs/test/dogs/"


## Process
split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [15]:
print(len(os.listdir(TRAINING_DOGS_DIR)))
print(len(os.listdir(TRAINING_CATS_DIR )))

12378
12391


In [18]:
## Check One Image
path = os.path.join(TRAINING_CATS_DIR, os.listdir(TRAINING_CATS_DIR)[100])
aimg = image.load_img(path)
aarr = image.img_to_array(aimg)
aarr.shape

(192, 196, 3)

In [20]:
## Basic Generators
# trn_datagen = ImageDataGenerator(rescale=1/255)
tst_datagen = image.ImageDataGenerator(rescale=1/255)

In [21]:
## Generators With Augmentation
trn_datagen = image.ImageDataGenerator(
                    rescale            = 1./255,
                    rotation_range     = 40,
                    width_shift_range  = 0.2,
                    height_shift_range = 0.2,
                    shear_range        = 0.2,
                    zoom_range         = 0.2,
                    horizontal_flip    = True,
                    fill_mode          = 'nearest')

In [22]:
trn_gen = trn_datagen.flow_from_directory(directory= "/tmp/cats-v-dogs/train",
                                          target_size = (150,150),
                                          batch_size  = 200, 
                                          class_mode  = 'binary')

val_gen = tst_datagen.flow_from_directory(directory= "/tmp/cats-v-dogs/test",
                                          target_size = (150,150),
                                          batch_size  = 200, 
                                          class_mode  = 'binary')

Found 24767 images belonging to 2 classes.
Found 4768 images belonging to 2 classes.


In [23]:
def d_cnn():
  
  l0 = layers.InputLayer((150,150,3))

  a1 = layers.Conv2D(32, (3,3), activation=activations.relu)
  a2 = layers.MaxPool2D((2,2))

  b1 = layers.Conv2D(64, (3,3), activation=activations.relu)
  b2 = layers.MaxPool2D((2,2))

  c1 = layers.Conv2D(128, (3,3), activation=activations.relu)
  c2 = layers.MaxPool2D((2,2))

  l1 = layers.Flatten()
  l2 = layers.Dropout(0.5)

  l3 = layers.Dense(512, activation=activations.swish)
  l4 = layers.Dense(128, activation=activations.swish)
  lf = layers.Dense(1, activation=activations.sigmoid)

  model = models.Sequential([l0, a1, a2, b1, b2, c1, c2, l1, l2, l3, l4, lf])

  return model

model = d_cnn()

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 36992)             0

In [25]:
model.compile(optimizer             ='rmsprop',
              loss                  = losses.binary_crossentropy,
              metrics               = ['accuracy'])

In [ ]:
history = model.fit(trn_gen,
                    epochs          = 10,
                    steps_per_epoch = 120,
                    validation_data = val_gen,
                    validation_steps= 10)

Epoch 1/10
 37/120 [========>.....................] - ETA: 1:55 - loss: 0.9873 - accuracy: 0.5439

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

120/120 [==============================] - 181s 2s/step - loss: 0.7804 - accuracy: 0.5709 - val_loss: 0.5946 - val_accuracy: 0.6875
Epoch 2/10
120/120 [==============================] - 178s 1s/step - loss: 0.6417 - accuracy: 0.6418 - val_loss: 0.5648 - val_accuracy: 0.7090
Epoch 3/10
120/120 [==============================] - 176s 1s/step - loss: 0.6052 - accuracy: 0.6700 - val_loss: 0.5023 - val_accuracy: 0.7480
Epoch 4/10
120/120 [==============================] - 174s 1s/step - loss: 0.5747 - accuracy: 0.6949 - val_loss: 0.4976 - val_accuracy: 0.7570
Epoch 5/10
120/120 [==============================] - 174s 1s/step - loss: 0.5552 - accuracy: 0.7122 - val_loss: 0.4596 - val_accuracy: 0.7950
Epoch 6/10
120/120 [==============================] - ETA: 0s - loss: 0.5369 - accuracy: 0.7285

In [ ]:
loss_acc_plots(history)